In [ ]:
import os, sys, json
import subprocess
import re
import pandas as pd
import time
import datetime

DIR = '/tmp/p'

: 

In [11]:
FAR_DATE = datetime.datetime(2021, 7, 20)
NEAR_DATE = datetime.datetime(2021, 9, 21)

def find_days_since_near_far(far_date, near_date):
    now = datetime.datetime.now()
    days_since_far_date = (now - far_date).days
    days_since_near_date = (now - near_date).days
    return days_since_far_date, days_since_near_date


FAR_DAYS, NEAR_DAYS = find_days_since_near_far(FAR_DATE, NEAR_DATE)


In [ ]:

def run_cmd(cmds):
    print(f'cmds = {cmds}')
    proc = subprocess.Popen(
        cmds,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    stdout, stderr = proc.communicate(timeout=1000)
    if stderr:
        stderr_s = stderr.decode('utf-8').split('\n')
        for s in stderr_s:
            print(f'stderr = {s}')
    return stdout.decode('utf-8')


In [ ]:


def get_kucoin_pairs():
    cmds = 've/bin/freqtrade list-markets --exchange kucoin --userdir freqtrade/user_data --quote USDT -1'.split(
        ' ')
    res = run_cmd(cmds).split('\n')
    return [
        r for r in res
        if r and not r.endswith('3L/USDT') and not r.endswith('3S/USDT')
    ]


In [14]:

def get_json_files(dir_path):
    return [
        os.path.join(dir_path, f)
        for f in os.listdir(dir_path)
        if f.endswith('.json')
    ]






In [ ]:
def read_json_file(json_file):
    with open(json_file, 'rt') as f:
        data = json.load(f)

    df = pd.DataFrame(data)
    df.columns = ['ts', 'open', 'high', 'low', 'close', 'volume']
    # df['date'] = df.ts.apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
    df['date'] = pd.to_datetime(df.ts, unit='ms')
    return df


def load_file_data(json_files):
    all_data = dict()
    for json_file in json_files:
        json_pd = read_json_file(json_file)
        token_name = json_file.split('/')[-1].split('.')[0]
        usdt_idx = token_name.find('_USDT')
        token_name_simplified = token_name[:usdt_idx]
        all_data[token_name_simplified] = json_pd
        print(f'token_name={token_name_simplified}; json_file={json_file}; json_pd = {json_pd}')
    return all_data



In [ ]:
def create_simple_relative_strength(token_name,df, btc_data, ref_date = FAR_DATE, rs_type_name='far', btc_ref='btc_far_ref'):
    # df['simple_rs']=  df['close'] / btc_data['close']
    first_date = df.date.min()
    if first_date > FAR_DATE:
        print(f'<{token_name}> first_date = {first_date}. Just released. Moving on')
        return None
    price_on_date = lambda d: d.loc[d['date'] == ref_date].iloc[0]['close']
    token_close_price_on_far_ref_date = price_on_date(df)
    btc_close_price_on_far_ref_date = price_on_date(btc_data)
    print(f'price_on_{ref_date} = {token_close_price_on_far_ref_date}; btc_close_price_on_far_ref_date={btc_close_price_on_far_ref_date}')
    df[rs_type_name] = df['close'] / token_close_price_on_far_ref_date
    df[btc_ref]  = btc_data['close'] / btc_close_price_on_far_ref_date
    df[rs_type_name + '_rs'] = df[rs_type_name] / df[btc_ref] - 1



In [ ]:


# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]


def download_data(kucoin_pairs):
    split_pairs = list(divide_chunks(kucoin_pairs, 50))
    for pairs_list in split_pairs:
        with open('/tmp/tmp-pairs.json', 'wt') as tmp_pairs_file:
            tmp_pairs_file.write(json.dumps(pairs_list))

        cmds = f'freqtrade download-data --exchange kucoin --userdir freqtrade/user_data --pairs-file /tmp/tmp-pairs.json --exchange kucoin --days {FAR_DAYS+20} -t 1d -d /tmp/p'.split(
            ' ')
        res = run_cmd(cmds)
        print(f'Downloading data for : {pairs_list}. \n\n res = {res}')
        print('Sleeping for 10 seconds')
        time.sleep(10)
        # print(res)


In [ ]:
def check_relative_strengths(pairs_to_use, btc_token_name):
    json_files = get_json_files(DIR)

    # print(f'json_files = {json_files}')
    find_json_file = lambda token: [j for j in json_files if token.replace('/', '_') in j][0]
    btc_json_file = find_json_file(btc_token_name)
    print(f'btc_json_file = {btc_json_file}')
    btc_json_data = read_json_file(btc_json_file)
    tokens_with_positive_far_rs = []
    tokens_with_positive_near_rs = []
    highlighted_tokens_with_positive_rs = []
    for token in pairs_to_use:
        if btc_token_name in token:
            continue
        print(f'token = {token}')
        token_json_file = find_json_file(token)
        print(f'  token_json_file = {token_json_file}')
        token_df = read_json_file(token_json_file)
        create_simple_relative_strength(token, token_df, btc_json_data, FAR_DATE,  rs_type_name='far', btc_ref='btc_far_ref')
        create_simple_relative_strength(token, token_df, btc_json_data, NEAR_DATE,  rs_type_name='near', btc_ref='btc_near_ref')
        last_row_far_rs = token_df.iloc[-1]['far_rs'] if 'far_rs' in token_df.columns else None
        last_row_near_rs = token_df.iloc[-1]['near_rs'] if 'near_rs' in token_df.columns else None

        first_date = token_df.date.min()
        if first_date <= FAR_DATE and last_row_far_rs is not None  and last_row_far_rs > 0 and last_row_near_rs > last_row_far_rs:
            token_df['highlight'] = 1
        else:
            token_df['highlight'] = 0


        last_row_highlight = token_df.iloc[-1]['highlight'] if 'highlight' in token_df.columns else None
        print(f'<{token}>: Far RS: {last_row_far_rs}; Near RS: {last_row_near_rs}; highlight: {last_row_highlight}')
        highlighted_tokens_with_positive_rs.append(token) if last_row_highlight == 1 else None

        if last_row_far_rs is not None and last_row_far_rs > 0:
            tokens_with_positive_far_rs.append(token)
            # print(f'<{token}>:  last_row_far_rs = {last_row_far_rs}')
            # print(f'<{token}>:  last_row_far_rs = {last_row_far_rs}')
        if last_row_near_rs is not None and last_row_near_rs > 0:
            tokens_with_positive_near_rs.append(token)
            # print(f'<{token}>:  last_row_near_rs = {last_row_near_rs}')
        # else:
        #     print(f'<{token}>:  last_row_near_rs = {last_row_near_rs}')

    return tokens_with_positive_far_rs, tokens_with_positive_near_rs, highlighted_tokens_with_positive_rs




In [ ]:
def main(should_download_pairs, should_download_data, should_calculate_relative_strength, debug):
    btc_token_name = 'BTC/USDT'
    # print(f'split_pairs = {split_pairs}; len = {len(split_pairs)}')
    if should_download_pairs:
        kucoin_pairs = get_kucoin_pairs()
        if debug:
            pairs_to_use = kucoin_pairs[:2] + [btc_token_name]
        else:
            pairs_to_use = kucoin_pairs
    else:
        pairs_to_use = ['1EARTH/USDT', '1INCH/USDT' , btc_token_name]
    print(f'kucoin_pairs = {pairs_to_use}')
    if should_download_data:
        print(f'Downloading data for: {pairs_to_use}')
        download_data(pairs_to_use)


    if should_calculate_relative_strength:
        tokens_with_positive_far_rs, tokens_with_positive_near_rs, highlighted_tokens_with_positive_rs = check_relative_strengths(pairs_to_use, btc_token_name)
        print(f'Tokens with positive relative strength FAR: {tokens_with_positive_far_rs}')
        print(f'Tokens with positive relative strength NEAR: {tokens_with_positive_near_rs}')
        print(f'highlighted_tokens_with_positive_rs = {highlighted_tokens_with_positive_rs}\n')
        tradingview_style = ','.join(['KUCOIN:' + x.replace('/', '') for x in highlighted_tokens_with_positive_rs])
        print(f'\nTokens with positive relative strength: {tradingview_style}\n')
        print(f'FAR_DAYS, NEAR_DAYS = {FAR_DAYS}, {NEAR_DAYS}')




main(should_download_pairs=True, should_download_data=False, should_calculate_relative_strength=True,  debug=False)